In [1]:
import sqlite3
import os
import cv2 as cv 
from PIL import Image

## **CREATE TABLE**
-----------------

In [32]:
def create_table(dirname, filename):
    #set paths and create dir where database is stored",
    database_path = os.path.join(os.pardir, dirname, filename)
    database_dir_path = os.path.dirname(database_path)

    if not os.path.exists(database_dir_path):
        os.makedirs(database_path)
    
    #create database connection
    conn = sqlite3.connect(database_path)
    curs = conn.cursor()
    
    create_table_statement = """CREATE TABLE if not exists images
                    (imageid INTEGER PRIMARY KEY,
                     filepath text not null,
                     filename text not null,
                     size text not null
                     )"""
    
    curs.execute(create_table_statement)
    
    conn.commit()

    conn.close()

## **FETCH DATA FROM HARDDRIVE**
----------------

In [34]:
#check path to harddrive,
root_dir = "F:\\data\\image_data"
if os.path.exists(root_dir):
    print(True)
else:
    print(False)

False


In [35]:
#get resolution
def get_image_size(image_path):
    with Image.open(image_path) as img:
        size= img.size
    return str(size)

In [36]:
#generator
def load_images_from_harddrive(root_dir):
    for root, dirs, files in os.walk(root_dir):
        print(len(files))
        for file in files:
            if file.lower().endswith((".jpg", ".png", ".jpeg")):
                size = get_image_size(os.path.join(root, file))
                yield root, file, size

## **INSERT DATA INTO DATABASE**
---------------

In [37]:
def main(root_dir, database_path, imageid=0):

    conn = sqlite3.connect(database_path)
    curs = conn.cursor()
    
    for root, file, size  in load_images_from_harddrive(root_dir):
        curs.execute("INSERT INTO IMAGES ('imageid', 'filepath', 'filename', 'size') VALUES (?,?,?,?)", (imageid, root, file, size))
        conn.commit()
        imageid += 1

        if imageid == 200:
            break

    conn.close()

In [38]:
def main_with_range(root_dir, database_path, range=100, imageid=0):

        conn = sqlite3.connect(database_path)
        curs = conn.cursor()
        while imageid > range:
                root, file, size = load_images_from_harddrive(root_dir)
                curs.execute("INSERT INTO IMAGES ('imageid', 'filepath', 'filename', 'size') VALUES (?,?,?,?)", (imageid, root, file, size))
                conn.commit()
                imageid += 1
        conn.close()

### **MAIN**
-------

In [39]:
root_dir = "F:\\data\\image_data"
database_path = os.path.join(os.pardir, "databases", "image_recommender.db")
database_dir_path = os.path.dirname(database_path)


create_table("databases", "image_recommender.db")
#main(root_dir)
main_with_range(root_dir, database_path, range=100)

## **TEST IF WORKED**
------

In [40]:
database_path = os.path.join(os.pardir, "databases", "image_recommender.db")
database_dir_path = os.path.dirname(database_path)
conn = sqlite3.connect(database_path)
curs = conn.cursor()

In [25]:
#delete all from table
curs.execute("DELETE FROM IMAGES")
conn.commit()

In [31]:
# select all 
curs.execute("SELECT * FROM IMAGES")
curs.fetchall()

[]

In [ ]:
#show database
def describe_table(curs, table_name, database_dir_path):

    # check if table is created
    print(f"Database created: {os.listdir(database_dir_path)}")


    curs.execute(f"PRAGMA table_info({table_name}")
    return curs.fetchall()

table_structure = describe_table(curs, "images")


In [ ]:
#delete table if needed
table_name = ["images"]
for name in table_name:
    curs.execute(f"DROP TABLE if exists {name}")

In [ ]:
#test to retrieve pictures from database",
curs.execute("SELECT filepath, filename FROM IMAGES")
firstpic = curs.fetchall()[0]
path = os.path.join(firstpic[0], firstpic[1])
image = Image.open(path)
image.show()